# Chat Driver

An OpenAI Chat Completions API wrapper.

## Notebook setup

Run this cell to set the notebook up. Other sections can be run independently.

In [10]:
%reload_ext autoreload
%autoreload 2

import os
from dotenv import load_dotenv
from azure.identity import aio, DefaultAzureCredential, get_bearer_token_provider, AzureCliCredential

from openai import AsyncAzureOpenAI, AzureOpenAI

import logging 
import json
from pathlib import Path

LOGGING = {
    "version": 1,
    "disable_existing_loggers": False,
    "formatters": {
        "json": {
            "()": "pythonjsonlogger.jsonlogger.JsonFormatter",
            "fmt": "%(asctime)s %(levelname)s %(name)s %(message)s",

        }
    },
}


# Set up structured logging to a file. All of the cells in this notebook use
# this logger. Find them at .data/logs.jsonl.
class JsonFormatter(logging.Formatter):
    def format(self, record) -> str:
        record_dict = record.__dict__
        log_record = {
            'timestamp': self.formatTime(record, self.datefmt),
            'level': record.levelname,
            'session_id': record_dict.get('session_id', None),
            'run_id': record_dict.get('run_id', None),
            'message': record.getMessage(),
            'data': record_dict.get('data', None),
            'module': record.module,
            'funcName': record.funcName,
            'lineNumber': record.lineno,
            'logger': record.name,
        }
        extra_fields = {
            key: value for key, value in record.__dict__.items() 
            if key not in ['levelname', 'msg', 'args', 'exc_info', 'funcName', 'module', 'lineno', 'name', 'message', 'asctime', 'session_id', 'run_id', 'data']
        }
        log_record.update(extra_fields)
        return json.dumps(log_record)

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
modules = ['httpcore.connection', 'httpcore.http11', 'httpcore.sync.connection', 'httpx', 'openai', 'urllib3.connectionpool', 'urllib3.util.retry']
for module in modules:
    logging.getLogger(module).setLevel(logging.ERROR)
if logger.hasHandlers():
    logger.handlers.clear()
data_dir = Path('.data')
if not data_dir.exists():
    data_dir.mkdir()
handler = logging.FileHandler(data_dir / 'logs.jsonl')
handler.setFormatter(JsonFormatter())
logger.addHandler(handler)


load_dotenv()
credential = DefaultAzureCredential()

azure_openai_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT", ""),
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT", ""),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION", ""),
    "max_retries": 2,
}
logger.info("Azure OpenAI configuration", extra=azure_openai_config)

async_client = AsyncAzureOpenAI(
    **azure_openai_config,
    azure_ad_token_provider=aio.get_bearer_token_provider(
        aio.AzureCliCredential(),
        "https://cognitiveservices.azure.com/.default",
    ),
)

client = AzureOpenAI(
    **azure_openai_config,
    azure_ad_token_provider=get_bearer_token_provider(
        AzureCliCredential(),
        "https://cognitiveservices.azure.com/.default",
    ),
)

model: str = azure_openai_config.get("azure_deployment", "gpt-4o")


## ChatCompletionsAPI usage

Azure/OpenAI's Chat Completions API is the fundamental building block of an AI assistant that uses the GPT model. 

- https://platform.openai.com/docs/api-reference/chat
- https://github.com/openai/openai-python/blob/main/api.md
- https://platform.openai.com/docs/api-reference/chat drivers

### Sync

In [3]:
completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model=model,
)
print(completion.model_dump_json(indent=2))

{'id': 'chatcmpl-AQga9wXFldlhHVce3CfXMQ7jXo9L9', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': 'This is a test.', 'refusal': None, 'role': 'assistant', 'function_call': None, 'tool_calls': None}}], 'created': 1730923577, 'model': 'gpt-4o-2024-08-06', 'object': 'chat.completion', 'service_tier': None, 'system_fingerprint': 'fp_d54531d9eb', 'usage': {'completion_tokens': 5, 'prompt_tokens': 12, 'total_tokens': 17, 'completion_tokens_details': None, 'prompt_tokens_details': None}}


### Async

In [4]:
message_event = await async_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model=model,
)
print(message_event)

ChatCompletion(id='chatcmpl-AQgaCid05i8dGgUHijQsvPAsMYbv4', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='This is a test.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1730923580, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_d54531d9eb', usage=CompletionUsage(completion_tokens=5, prompt_tokens=12, total_tokens=17, completion_tokens_details=None, prompt_tokens_details=None))


### Streaming

In [5]:
stream = await async_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model=model,
    stream=True,
)
async for chunk in stream:
    print(chunk.model_dump())

{'id': 'chatcmpl-AQgaEhy4ed19h72VpBu8nJBJE3g3L', 'choices': [{'delta': {'content': '', 'function_call': None, 'refusal': None, 'role': 'assistant', 'tool_calls': None}, 'finish_reason': None, 'index': 0, 'logprobs': None}], 'created': 1730923582, 'model': 'gpt-4o-2024-08-06', 'object': 'chat.completion.chunk', 'service_tier': None, 'system_fingerprint': 'fp_d54531d9eb', 'usage': None}
{'id': 'chatcmpl-AQgaEhy4ed19h72VpBu8nJBJE3g3L', 'choices': [{'delta': {'content': 'This', 'function_call': None, 'refusal': None, 'role': None, 'tool_calls': None}, 'finish_reason': None, 'index': 0, 'logprobs': None}], 'created': 1730923582, 'model': 'gpt-4o-2024-08-06', 'object': 'chat.completion.chunk', 'service_tier': None, 'system_fingerprint': 'fp_d54531d9eb', 'usage': None}
{'id': 'chatcmpl-AQgaEhy4ed19h72VpBu8nJBJE3g3L', 'choices': [{'delta': {'content': ' is', 'function_call': None, 'refusal': None, 'role': None, 'tool_calls': None}, 'finish_reason': None, 'index': 0, 'logprobs': None}], 'create

## OpenAI Helpers

## Standardized response handling

In [11]:
from context import Context
from openai_client.errors import CompletionError, validate_completion
from openai_client.logging import extra_data, serializable_completion_args
from openai_client.completion import completion_message_string

context = Context("conversation-id-1005")

# We use a metadata dictionary in our helpers to store information about the
# completion request.
metadata = {}

# This is just standard OpenAI completion request arguments.
completion_args = {
    "model": model,
    "messages": [
        {
            "role": "system",
            "content": "You are a famous computer scientist. You are giving a talk at a conference. You are talking about the future of AI and how it will change the world. You are asked a questions by audience members and answer thoughtfully.",
        },
        {
            "role": "user",
            "content": "What is the future of AI?",
        }
    ],
}

# If we these completion args to logs and metadata, though, they need to be
# serializable. We have a helper for that.
metadata["completion_args"] = serializable_completion_args(completion_args)

# We have helpers for validating the response and handling exceptions in a
# standardized way. These ensure that logging happens and metadata is loaded up
# properly.
try:
    completion = await async_client.beta.chat.completions.parse(**completion_args)

    # This helper looks for any error-like situations (the model refuses to
    # answer, content filters, incomplete responses) and throws exceptions that
    # are handled by the next helper. The first argument is an identifier that
    # will be used for logs and metadata namespacing.
    validate_completion(completion)
    logger.debug("completion response.", extra=extra_data(completion))
    metadata["completion"] = completion.model_dump()

except Exception as e:
    # This helper processes all the types of error conditions you might get from
    # the OpenAI API in a standardized way.
    completion_error = CompletionError(e)
    print(completion_error)
    print(completion_error.body)

else:
    # The message_string helper is used to extract the response from the completion
    # (which can get tedious).
    print(completion_message_string(completion))


The future of AI is both exciting and complex, with its trajectory shaped by advances in technology, ethical considerations, and societal needs. As we look ahead, several key themes emerge:

1. **Integration and Personalization**: AI will become increasingly integrated into our daily lives, driving personalized experiences. From healthcare to education, AI systems will tailor recommendations and interventions to individual needs, optimizing outcomes across various sectors.

2. **AI in Healthcare**: We can expect AI to revolutionize the healthcare industry by enhancing diagnostic accuracy, streamlining administrative processes, and developing personalized medicine. AI-driven tools could predict diseases, recommend treatments, and even aid in surgical procedures, ultimately improving patient care and reducing costs.

3. **Autonomous Systems**: Autonomous vehicles, drones, and robotic systems will become more prevalent, transforming industries like transportation, logistics, and manufactu

### JSON

In [12]:
from context import Context
from openai_client.errors import CompletionError, validate_completion
from openai_client.logging import extra_data, serializable_completion_args
from openai_client.completion import completion_message_dict, JSON_OBJECT_RESPONSE_FORMAT

context = Context("conversation-id-1002")
metadata = {}
completion_args = {
    "model": model,
    "messages": [
        {
            "role": "system",
            "content": "You are a famous computer scientist. You are giving a talk at a conference. You are talking about the future of AI and how it will change the world. You are asked a questions by audience members and return your answer as valid JSON like { \"thoughts\": <some thoughts>, \"answer\": <an answer> }.",
        },
        {
            "role": "user",
            "content": "What is the future of AI?",
        }
    ],
    "response_format": JSON_OBJECT_RESPONSE_FORMAT,
}
metadata["completion_args"] = serializable_completion_args(completion_args)
try:
    completion = await async_client.beta.chat.completions.parse(**completion_args)
    validate_completion(completion)
    metadata["completion"] = completion.model_dump()
except Exception as e:
    completion_error = CompletionError(e)
    metadata["completion_error"] = completion_error.body
    logger.error(completion_error.message, extra=extra_data({"error": completion_error.body, "metadata": metadata}))
else:
    message = completion_message_dict(completion)
    print(json.dumps(message, indent=2))


{
  "thoughts": "AI is rapidly evolving and has the potential to transform virtually every industry. With advancements in machine learning, natural language processing, and robotics, AI will continue to play a crucial role in automating routine tasks and providing intelligent insights.",
  "answer": "The future of AI involves deeper integration into everyday life and industry. We will likely see AI systems becoming more autonomous, sophisticated, and seamlessly integrated into systems like healthcare, transportation, and personalized services. AI's future encompasses not only technological advancements but also ethical considerations and ensuring beneficial societal impacts."
}


### Structured Output

Any Pydantic BaseModel can be used as the "response_format" and OpenAI will try to load it up for you.

In [13]:
from context import Context
from pydantic import BaseModel
from typing import cast
from openai_client.errors import CompletionError, validate_completion
from openai_client.logging import extra_data, serializable_completion_args
from openai_client.completion import completion_message_dict, JSON_OBJECT_RESPONSE_FORMAT

class Output(BaseModel):
    thoughts: str
    answer: str

context = Context("conversation-id-1002")
metadata = {}
completion_args = {
    "model": model,
    "messages": [
        {
            "role": "system",
            "content": "You are a famous computer scientist. You are giving a talk at a conference. You are talking about the future of AI and how it will change the world. You are asked a questions by audience members and return your answer as valid JSON like { \"thoughts\": <some thoughts>, \"answer\": <an answer> }.",
        },
        {
            "role": "user",
            "content": "What is the future of AI?",
        }
    ],
    "response_format": Output,
}

metadata["completion_args"] = serializable_completion_args(completion_args)
try:
    completion = await async_client.beta.chat.completions.parse(**completion_args)
    validate_completion(completion)
    metadata["completion"] = completion.model_dump()
except Exception as e:
    completion_error = CompletionError(e)
    metadata["completion_error"] = completion_error.body
    logger.error(completion_error.message, extra=extra_data({"error": completion_error.body, "metadata": metadata}))
else:
    # The parsed message is in the `parsed` attribute.
    output = cast(Output, completion.choices[0].message.parsed)
    print(output.model_dump_json(indent=2))

    # Or you can just get the text of the message like usual.
    # print(completion.choices[0].message.content)


{
  "thoughts": "AI has shown immense potential in various fields, including healthcare, transportation, and climate science. The trajectory of its development suggests even greater integration into daily life and business operations.",
  "answer": "The future of AI lies in its ability to become more autonomous, adaptive, and integrated into various facets of life and industry. We will likely see AI taking on increasingly complex tasks with minimal human intervention, improving efficiency and productivity. It will transform sectors like healthcare through predictive diagnostics, personalize education, and enhance decision-making in businesses with deeper insights from data analytics. However, it will also be crucial to address ethical concerns, such as privacy, bias, and the impact on jobs, to ensure AI serves the broader good."
}


## OpenAI Chat Completion Driver (a.k.a "chat driver")

### OpenAI Assistants

The Azure/OpenAI Assistants API is newer, stateful API that splits an `assistant` from the data about a conversation `thread` that can be `run` against an `assistant`. Additionally, you can add `tools` to an assistant that enable the assistant to have more interactive capabilities. The tools currently available are:

- *Functions*: Registering local functions with the assistant so it knows it can call them before generating a response. This is a "hold on let me look that up for you" kind of interaction.
- *File Search* (formerly the retrieval plugin): Attach one or more files and they will be RAG-vectorized and available as content to the assistant.
- *Code Interpreter*: Run python code in a secure sandbox.

The Assistant API productized as OpenAI's `GPTs` product. The `GPT Builder` lets developers create and deploy GPTs assistants using a web interface.

### Chat Driver

But an "assistant" requires pretty strong "abstraction lock-in". This thing isn't really an assistant in the fullest sense... it's more like a "pseudo-assistant", but this confuses things. Let's just let the Chat Completion API be what it is and drive it as necessary as we create our assistants. Let's just wrap up the function calling bits (which, ultimately, can give you the other tools like Functions and File Search) in a simple-to-use GPT-like interface we'll call a *chat driver*.

The chat driver is meant to be used the exact way the Chat Completions API is... just easier.

Our chat driver provides:

- The ability to almost magically register functions to the function tool using a `FunctionRegistry`.
- Tracking of message history.
- Management of a `Context` object that can be used for session management and supply additional context to functions.
- Some prompt creation helpers.
- Other utilities... this is just meant to be an interface you can use to forget about all the api complexities.

#### Here is the simplest usage of a chat driver

Notice that a .data directory is created by default. This is where the conversation history is stored.

In [6]:
from chat_driver import ChatDriver, ChatDriverConfig
from context import Context

# When an chat driver is created, it will automatically create a context with a
# session_id. Or, if you want to use a specific session_id, you can pass it as
# an argument. This is useful for scoping this chat driver instance to an
# external identifier.
context = Context("simple-id-1000")

instructions = "You are a famous computer scientist. You are giving a talk at a conference. You are talking about the future of AI and how it will change the world. You are asked a questions by audience members."

chat_driver = ChatDriver(
    ChatDriverConfig(
        openai_client=async_client,
        model=model,
        instructions=instructions,
        context=context,
    ),
)

message_event = await chat_driver.respond("What is the future of AI?")
print(message_event.model_dump_json(indent=2))

conversation-id-1000

Hello Paul! How can I assist you today?

Commands:
help(): Return this help message.
erase(name: str): Erases a stored value.
echo(text: str): Return the text.
get_file_contents(file_path: str): Return the contents of a file.

conversation-id-1000: Hi, my name is Paul.

The content of "123.txt" is: "The purpose of life is to be happy."

{
  "id": "9444f8f2-aa5a-4c19-9bf4-d36b6bfa9ba4",
  "session_id": null,
  "timestamp": "2024-11-06T20:06:27.777169",
  "message": "The content of \"123.txt\" is: \"The purpose of life is to be happy.\"",
  "metadata": {
    "completion_args": {
      "model": "gpt-4o",
      "messages": [
        {
          "role": "user",
          "content": "What is the future of AI?"
        },
        {
          "role": "assistant",
          "content": "The future of AI is incredibly promising and will likely be revolutionary in many aspects of our lives. In the coming years and decades, we can expect several major trends and developments t

#### You can tell the chat driver to always return JSON

Note: You MUST include the word "JSON" somewhere in your instructions. This is an OpenAI requirement for JSON return. However, note that a ChatDriver response method only returns string responses in its returned `MessageEvent` since these events are intended to be used in chat scenarios. If you want to transform it into an actual JSON object, you'll need to `json.loads(response.message)` it yourself.

In [14]:

from chat_driver import ChatDriver, JSON_OBJECT_RESPONSE_FORMAT, ChatDriverConfig
from context import Context

context = Context("conversation-id-1002")
instructions = 'You are a famous computer scientist. You are giving a talk at a conference. You are talking about the future of AI and how it will change the world. You are asked a questions by audience members and return your answer as valid json, like this: { "thoughts": <some thoughts>, "answer": <an answer> }.'

chat_driver = ChatDriver(
    ChatDriverConfig(
        openai_client=async_client,
        model=model,
        instructions=instructions,
        context=context,
    ),
)

message_event = await chat_driver.respond("What is the future of AI?", response_format=JSON_OBJECT_RESPONSE_FORMAT)
print(message_event.message)

print(json.dumps(json.loads(message_event.message or ""), indent=2))


{ "thoughts": "The future of AI is a topic filled with both excitement and caution, as it holds the potential for transformative impacts across various facets of society.", "answer": "The future of AI is likely to involve increased integration into everyday life, with advancements in areas such as natural language processing, computer vision, and data analysis. AI is set to enhance fields like healthcare through predictive diagnostics, personalized medicine, and robotic surgery. In transportation, autonomous vehicles will likely become more common. AI will also improve efficiencies in industries like manufacturing and logistics. However, this future will also require addressing ethical considerations, data privacy concerns, and ensuring that AI technologies are accessible and beneficial for all members of society. A key focus will be on developing fair and unbiased AI systems, and creating governance frameworks that promote responsible AI development and usage." }
{
  "thoughts": "The 

#### Chat drivers can return structured responses

Just give it a class that inherits from Pydantic BaseModel and it will return your response in that structure. Most the time, if you want this kind of thing, you probably don't want a Chat Driver. Just use the OpenAI client with the helpers above. However, in the rare case that you want to actually use the structured output to guarantee the response from the model, this is here for you. You'll need to marshal the object yourself.

In [ ]:
# Simple chat driver w/ structured response

from typing import cast
from chat_driver import ChatDriver, ChatDriverConfig, LocalMessageHistoryProvider
from context import Context
from pydantic import BaseModel

context = Context("structured-1000")

instructions = 'You are a famous computer scientist. You are giving a talk at a conference. You are talking about the future of AI and how it will change the world. You are asked a questions by audience members and return a thoughtful response.'

class ThoughtfulResponse(BaseModel):
    """A thoughtful response to a question."""
    thoughts: str
    answer: str

chat_driver = ChatDriver(
    ChatDriverConfig(
        openai_client=async_client,
        model=model,
        instructions=instructions,
        context=context,
    ),
)

# Let's clear the data from previous runs.
message_provider = cast(LocalMessageHistoryProvider, chat_driver.message_provider)
message_provider.delete_all()

message_event = await chat_driver.respond("What is the future of AI?", response_format=ThoughtfulResponse)

# As always, the event will come back with a string message.
print(message_event.message)

# If you really want to, you can turn the JSON string back into a Pydantic model.
thoughtful_response = ThoughtfulResponse(**json.loads(message_event.message or ""))
print(thoughtful_response.model_dump_json(indent=2))

{"thoughts":"AI is rapidly advancing and its future holds great potential and considerable responsibility. As AI technologies become more integrated into society, we will see transformations across various sectors, including healthcare, transportation, education, and manufacturing. However, these advancements also come with ethical considerations and the need for responsible development.","answer":"The future of AI is both exciting and complex. We will likely see AI becoming more autonomous, improving its ability to understand and interpret nuanced human interactions, and solving complex problems in ways that were previously unimaginable. In healthcare, AI could lead to personalized medicine and better diagnostic tools. In transportation, we'll see more autonomous vehicles that could reshape our cities and daily commutes. However, with these advancements comes the responsibility to address ethical concerns such as privacy, security, and fairness. It's crucial that as AI capabilities gr

#### You can register functions to chat drivers

Chat drivers will use any functions you give it as both OpenAI tool calls, and as commands.

With each response call, you can specify what type of response you want to have... string, dictionary, or Pydantic model.

In [ ]:
from typing import Any, cast
from chat_driver import ChatDriver, ChatDriverConfig
from context import Context, ContextProtocol
from chat_driver import LocalMessageHistoryProvider
from pydantic import BaseModel, Field


# When an chat driver is created, it will automatically create a context with a
# session_id. Or, if you want to use a specific session_id, you can pass it as
# an argument. This is useful for scoping this chat driver instance to an
# external identifier.
context = Context("conversation-id-1002")


# Define tool functions for the chat driver. All functions used by the chat driver
# require a session_id as the first argument.
def get_file_contents(context: Context, file_path: str) -> str:
    """
    Return the contents of a file.

    Args:
    - file_path: The path to the file.
    """
    return "The purpose of life is to be happy."


def erase(context: Context, name: str) -> str:
    """Erases a stored value."""
    return f"{context.session_id}: {name} erased"

def json_thing(context: Context) -> dict[str, Any]:
    """Return json."""
    return {"key": "value"}

class Input(BaseModel):
    zipcode: str

class Weather(BaseModel):
    description: str = Field(description="The weather description.")
    cloud_cover: float
    temp_c: float
    temp_f: float

def get_weather(context: Context, input: Input) -> Weather:
    """Return the weather."""
    return Weather(description="Sunny", cloud_cover=0.2, temp_c=25.0, temp_f=77.0)

# Define the chat driver.
instructions = "You are a helpful assistant."

# Define the conversation so far (optional).
# messages: List[ChatCompletionMessageParam] = []
# localMessageHistoryConfig = LocalMessageHistoryProviderConfig(f"./data/{context.session_id}", messages)
# message_provider = LocalMessageHistoryProvider(localMessageHistoryConfig)

chat_driver = ChatDriver(
    ChatDriverConfig(
        openai_client=async_client,
        model=model,
        instructions=instructions,
        context=context,
        # message_provider=message_provider,
        commands=[erase, json_thing, get_weather],  # Commands can be registered when instantiating the chat driver.
        functions=[erase, json_thing, get_weather],  # Functions can be registered when instantiating the chat driver.
    ),
)

# Let's clear the data from previous runs.
message_provider = cast(LocalMessageHistoryProvider, chat_driver.message_provider)
message_provider.delete_all()


# You can also use the `register_function` decorator to register a function.
# Remember, all functions used by the chat driver require a session_id as the
# first argument.
@chat_driver.register_function_and_command
def echo(context: ContextProtocol, text: str) -> str:
    """Return the text."""
    return f"{context.session_id}: {text}"


# You can also register functions manually.
chat_driver.register_function_and_command(get_file_contents)

# Ok. Let's see if we got one.
print(chat_driver.context.session_id)

# Let's see if the agent can respond.
message_event = await chat_driver.respond("Hi, my name is Paul.")
print()
print(message_event.message)

# Help command (shows command available).
message_event = await chat_driver.respond("/help")
print()
print(message_event.message)

# We can run any function or command directly.
message_event = await chat_driver.functions.echo("Echo this.")
print()
print(message_event)

# Let's see if the chat driver has the ability to run it's own registered function.
message_event = await chat_driver.respond("Please tell me what's in file 123.txt.")
print()
print(message_event.message)

# Stuctured output.
message_event = await chat_driver.respond("What is the weather in 90210?", response_format=Weather)
print()
print(message_event.message)

# Let's see the full response event.
# print()
# print(response.to_json())

conversation-id-1002

Hello Paul! How can I assist you today?

Commands:
help(): Return this help message.
erase(name: str): Erases a stored value.
json_thing(): Return json.
get_weather(input: Input): Return the weather.
echo(text: str): Return the text.
get_file_contents(file_path: str): Return the contents of a file.

Args:
- file_path: The path to the file.

conversation-id-1002: Echo this.

The content of "123.txt" is: "The purpose of life is to be happy."

{"description":"Sunny","cloud_cover":0.2,"temp_c":25.0,"temp_f":77.0}


## Chat with a chat driver

In [7]:
from chat_driver import ChatDriverConfig, ChatDriver
from context import Context

context = Context("conversation-id-1006")


def get_file_contents(context: Context, file_path: str) -> str:
    """Returns the contents of a file."""
    return "The purpose of life is to be happy."


def erase(context: Context, name: str) -> str:
    """Erases a stored value."""
    return f"{context.session_id}: {name} erased"


def echo(context: Context, value: Any) -> str:  # noqa: F811
    """Echos a value as a string."""
    match value:
        case str():
            return value
        case list():
            return ", ".join(map(str, value))
        case dict():
            return json.dumps(value)
        case int() | bool() | float():
            return str(value)
        case _:
            return str(value)


functions = [get_file_contents, erase, echo]

# Define the chat driver.
chat_driver_config = ChatDriverConfig(
    openai_client=async_client,
    model=model,
    instructions="You are an assistant that has access to a sand-boxed Posix shell.",
    context=context,
    commands=functions,
    functions=functions,
)

chat_driver = ChatDriver(chat_driver_config)

# Note: Look in the .data directory for the logs, message history, and other data.

# Chat with the skill.
while True:
    message = input("User: ")
    if message == "":
        break
    print(f"User: {message}", flush=True)
    message_event = await chat_driver.respond(message)
    if message_event.metadata.get("error"):
        print(f"Error: {message_event.metadata.get('error')}")
        print(message_event.to_json())
        continue
    # You can print the entire message event! 
    # print(response.to_json())
    print(f"Assistant: {message_event.message}", flush=True)

User: /echo("hello world")
Assistant: hello world


## Chat Driver with an Assistant Drive

In [ ]:
from io import BytesIO
from typing import Any, BinaryIO
from chat_driver import ChatDriverConfig, ChatDriver, ChatDriverConfig
from context import Context
from assistant_drive import Drive, DriveConfig, IfDriveFileExistsBehavior 

session_id = "conversation-id-1001"

context = Context(session_id)

def get_drive_from_context(context):
    return Drive(DriveConfig(root=f".data/drive/{context.session_id}"))

def write_file_contents(context: Context, file_path: str, contents: str) -> str:
    """Writes the contents to a file."""
    drive = get_drive_from_context(context)
    content_bytes: BinaryIO = BytesIO(contents.encode("utf-8"))
    drive.write(content_bytes, file_path, if_exists=IfDriveFileExistsBehavior.OVERWRITE)
    return f"{file_path} updated."

def read_file_contents(context: Context, file_path: str) -> str:
    """Returns the contents of a file."""
    drive = get_drive_from_context(context)
    with drive.open_file(file_path) as file:
        return file.read().decode("utf-8")

functions = [write_file_contents, read_file_contents]

# Define the chat driver.
chat_driver_config = ChatDriverConfig(
    openai_client=async_client,
    model=model,
    instructions="You are an assistant that has access to a sand-boxed Posix shell.",
    context=context,
    commands=functions,
    functions=functions,
)

chat_driver = ChatDriver(chat_driver_config)

# Note: Look in the .data directory for the logs, message history, and other data.

# Chat with the skill.
while True:
    message = input("User: ")
    if message == "":
        break
    print(f"User: {message}", flush=True)
    message_event = await chat_driver.respond(message)
    # You can print the entire response event! 
    # print(response.to_json())
    print(f"Assistant: {message_event.message}", flush=True)